In [79]:
import seaborn as sns
import pandas as pd
import dataframe_image
import datetime
import matplotlib.pyplot as plt
from datetime import timedelta

In [80]:
def openFile(path):
    data = pd.read_csv(path)
    return data

In [95]:
def cleanUp (data, process, completedKeyword, dropColumns=['Resource', 'Loaded','LastUpdated', 'Exception', 'Deferred','Completed','Priority','Attempt','Locked','ExceptionReason','isLocked']):
    data['isLocked'] = data['Locked'].apply(lambda x: 1 if pd.notnull(x) else 0)
    data['Status'] = data['Status'].apply(lambda x: x if pd.notnull(x) else "Pending")
    data = data.apply(lambda row: setStatus(row, completedKeyword), axis=1)
    data = data.drop(columns=dropColumns)
    data["Process"] = process
    return data

In [82]:
def toString(data):
    print("Complete: " + getStatusCount(data,"Complete"))
    print("Business Exception: " + getStatusCount(data,"Business Exception"))
    print("Locked: " + getStatusCount(data,"Locked"))
    print("Pending " + getStatusCount(data,"Pending"))
    print("Exception " + getStatusCount(data,"Exception"))

In [83]:
def setStatus(row, completedKeyword):
    if pd.notnull(row["Tags"]) and "Business Exception" in row["Tags"]:
        row["Status"] = "Business Exception"
    elif row["isLocked"] == 1:
        row["Status"] = "Locked"
    elif row["Status"] == completedKeyword:
        row["Status"] = "Complete"
    elif row["Status"] == "Pending":
        pass
    else:
        row["Status"] = "Exception"
    return row

In [84]:
def getStatusCount(data,status):
    count, _ = data[data["Status"] == status].shape
    return str(count)

In [85]:
def getWorktimeDF(row):
    minutes = int(row["Worktime"]/60)
    seconds = (row["Worktime"]%60)
    timespan = timedelta(minutes=minutes,seconds=seconds)
    row["Worktime"] = timespan
    return row

In [86]:
def getWorktime(value):
    minutes = int(value/60)
    seconds = (value%60)
    timespan = timedelta(minutes=minutes,seconds=seconds)
    return timespan

In [87]:
def averageWorktime(data,status=None):
    if status is None:
        return round(data["Worktime"].mean())
    else:
        return round(data[data["Status"] == status]["Worktime"].mean())

In [96]:
directory = "resources/reports/08-26/"
filename = "icd.csv"
path = directory + filename
data = openFile(path)
cleandf = cleanUp(data,"ICD","Complete")
cleandf.shape

(3797, 6)

In [97]:
cleandf.head()

,Id,KeyValue,Status,Tags,Worktime,Process
0,d85107f7-1863-4498-adf6-ed899104fcf7,be412b78-0379-4f05-a5eb-5ccdb5978730,Locked,Conventional,0,ICD
1,de1f84fb-3f83-4be2-8e57-262485faeaa2,6bc7c28b-7e75-4dfe-a8c2-be8b19cb0159,Complete,Redisclosure: 0,632,ICD
2,2795cd33-e771-420d-bcce-4c8f33feeaab,6450fe23-ddad-41e4-b056-06da2c75a292,Locked,NaN,0,ICD
3,b0e8d75f-786c-4c9a-87c1-438783443aa1,6f96d1ce-a756-4387-8f55-29aa886c40c1,Complete,Conventional;Redisclosure: 1;Run UCD,718,ICD
4,ec2fe0f5-b498-4396-a724-aeb70611551a,6dc44fd3-fb9f-4cee-b69a-22905ce657f9,Locked,Conventional,0,ICD


In [116]:
df = cleandf.groupby('Id').last().reset_index()
total, _ = df.shape

(3107, 6)

In [113]:
complete = df['Status'].apply(lambda status: status == "Complete").sum()
exception = df['Status'].apply(lambda status: status == "Exception").sum()
business = df['Status'].apply(lambda status: status == "Business Exception").sum()

print("Complete = "+str(complete))
print("Business = "+str(business))
print("Exception = "+str(exception))

Complete = 2432
Business = 103
Exception = 510


In [122]:
completionRate = ((complete+business)/total)*100
completion

np.float64(86.5683434290229)

In [121]:
all = total - exception
completion = (all/total)*100
completion

np.float64(86.5683434290229)